In [1]:
import pandas as pd
import numpy as np
import pickle
import math

In [2]:
file_to_read = open("user2movie.json", "rb")
user2movies = pickle.load(file_to_read)

file_to_read = open("movie2user.json", "rb")
movie2users = pickle.load(file_to_read)

file_to_read = open("usermovie2rating.json", "rb")
usermovie2rating = pickle.load(file_to_read)

file_to_read = open("usermovie2rating_test.json", "rb")
usermovie2rating_test = pickle.load(file_to_read)

In [3]:
def user_avg_rating(user_id):
    count_rating =0
    sum_rating = 0
    for movies in user2movies[user_id]:#iterating over all user id rated movies
        sum_rating += usermovie2rating[(user_id, movies)]#sum the rating
        count_rating+=1
    avg_rating = sum_rating/count_rating
    return avg_rating

    

In [4]:
def calc_single_w(user_a, user_b):
#getting two user's id, returning the weight by peerson
    user_a_mean = user_avg_rating(user_a)#R_a- can shorten...calc avg for all user in advanced
    user_b_mean = user_avg_rating(user_b)#R_b
    numerator = 0
    denominator = 0
    dom_a = 0
    dom_b = 0
    #finding the subset of intersection between movies rated by a and b
    set_a = user2movies[user_a]
    set_b = user2movies[user_b]
    intersection = set(set(set_a) & set(set_b))
    if len(intersection)<5:#if intersection to low, ignore this user
        return 0
    for movie in intersection:
        movie_rating_a = usermovie2rating[(user_a, movie)]
        movie_rating_b = usermovie2rating[(user_b, movie)]
        movie_dev_a = movie_rating_a - user_a_mean#user a rating deviation of movie
        movie_dev_b = movie_rating_b - user_b_mean#user b rating deviation of movie
        numerator += movie_dev_a*movie_dev_b
        dom_a += movie_dev_a**2
        dom_b += movie_dev_b**2
    dom_a = math.sqrt(dom_a)
    dom_b = math.sqrt(dom_b)
    denominator = dom_a*dom_b
    w = numerator/denominator
    return w
    

In [5]:

def calculate_all_w():
    #calculating all the weights, putting into dict
    user_user_w = {}

    for user_i in user2movies.keys():#running over all users
        for user_j in user2movies.keys():
            if user_i == user_j:
                continue
            if (user_j, user_i) in user_user_w:
                user_user_w[(user_i, user_j)] = user_user_w[(user_j, user_i)]#correlation is symetric so if already in dict no need to calc again
            else:
                 user_user_w[(user_i, user_j)] = calc_single_w(user_i, user_j)
        
    return user_user_w



In [6]:
from heapq import nlargest
def neighborfy(user_id, user_user_w):
#should get a subset of users who rated the movie to k top user's weights to our user
   
    k=40
    userid_user_w = {}
    for user in user2movies.keys():#looping throw all users
        if user == user_id:
            continue
        else:
            userid_user_w[user] = abs(user_user_w[user_id, user])#taking the two users weight to the new user_id dict
    
    neighbors = nlargest(k, userid_user_w, key = userid_user_w.get)#geeting the k best neighbors
    return neighbors



In [7]:
def get_score(user_id, movie_id, user_user_w, neighborhood, user_avg):

    user_rated_item_full= movie2users[movie_id]# the subset of users who rated the item movie_id 
    user_rated_item = set(set(user_rated_item_full) & set(neighborhood))#the intersection from the userid neighbours
    w = {}# a dict containing all the wheights between user_id and other users from his neighbors who rated the item
    userId_mean = user_avg[user_id]#user_id avg rating of all movies
    #TO DO- calc in advance the rating- not depended on movie!
    numerator = 0
    denominator = 0
    for user in user_rated_item:
        w[user]= user_user_w[(user_id, user)]# fetch current user and user_id weight. 
        user_rating= user_avg[user]#get current user avg
        deviation = user_rating - userId_mean
        numerator+= w[user]*deviation
        denominator = denominator+abs(w[user])
    
    
    if denominator == 0:
        return -10000  
    score = userId_mean+ (numerator/denominator)
    return score
        


In [8]:
def calc_mse(test, pred):
    error =0
    count = 0
    for pair in test.keys():
        error += (test[pair]-pred[pair])**2
        count+=1
    mse = error/count
    return mse


In [9]:
#this is the call for w calculation- it takes 175 min. so we ran it once and saved as file.
#w = calculate_all_w()#calculate w on train dicts
#with open('weights.json', 'wb') as f:
#  pickle.dump(w, f)

In [10]:
file_to_read = open("weights.json", "rb")
w = pickle.load(file_to_read)

{(2462, 1516): 0.30933031194953775,
 (2462, 204): 0.43329169268461915,
 (2462, 1654): 0.1215552802888242,
 (2462, 3618): 0.4465840733967296,
 (2462, 1335): 0.17201875493742808,
 (2462, 2279): 0.37782666478718385,
 (2462, 3266): 0.15873673223163798,
 (2462, 64): 0.218060771358824,
 (2462, 3809): 0.30718501620311145,
 (2462, 387): 0.40013604971239813,
 (2462, 1976): 0.37275187103397045,
 (2462, 1782): 0.13757439770478813,
 (2462, 3039): 0.4619122497614955,
 (2462, 2628): 0.17064270231702516,
 (2462, 2129): 0.2942593928258016,
 (2462, 3078): 0.3097426535759411,
 (2462, 3221): 0.13096051909145878,
 (2462, 616): 0.33524481012718593,
 (2462, 1376): 0.3111517043754545,
 (2462, 1549): 0.022660214823421718,
 (2462, 4847): 0.41326039485450106,
 (2462, 4929): 0.5470359584780725,
 (2462, 3210): 0.2759342908494684,
 (2462, 4108): 0.4246426962981356,
 (2462, 1193): 0.30913926161017613,
 (2462, 341): 0.5743822502605404,
 (2462, 3204): 0.2086489921642712,
 (2462, 2912): 0.5818580232006478,
 (2462, 232

In [12]:
#calc all users avg and neighbors
user_avg = {}
neighborhood = {}
for user in user2movies.keys():
    neighborhood[user] = neighborfy(user, w)#lets find his neiborhood
    user_avg[user] = user_avg_rating(user)


In [13]:

scores = {}
i=0
errors = []
for pair in usermovie2rating_test.keys():
    user = pair[0]   
    scores[pair] = get_score(pair[0], pair[1], w, neighborhood[pair[0]], user_avg)#for every pair in the test lets find the score
    if scores[pair] == -10000:
        scores.pop(pair)
        errors.append(pair)
        print("error calculating %s prediction" %(pair,))
    i+=1


error calculating (2939, 609) prediction
error calculating (426, 284) prediction
error calculating (4644, 884) prediction
error calculating (2119, 290) prediction
error calculating (4107, 403) prediction
error calculating (1841, 91) prediction
error calculating (2961, 817) prediction
error calculating (2663, 598) prediction
error calculating (4031, 442) prediction
error calculating (3437, 190) prediction
error calculating (448, 753) prediction
error calculating (853, 178) prediction
error calculating (4807, 577) prediction
error calculating (4807, 669) prediction
error calculating (4738, 944) prediction
error calculating (1344, 459) prediction
error calculating (3393, 529) prediction
error calculating (1602, 839) prediction
error calculating (2073, 999) prediction
error calculating (552, 408) prediction
error calculating (797, 649) prediction
error calculating (4518, 740) prediction
error calculating (2128, 148) prediction
error calculating (4148, 137) prediction
error calculating (255

In [14]:
errors

[(2939, 609),
 (426, 284),
 (4644, 884),
 (2119, 290),
 (4107, 403),
 (1841, 91),
 (2961, 817),
 (2663, 598),
 (4031, 442),
 (3437, 190),
 (448, 753),
 (853, 178),
 (4807, 577),
 (4807, 669),
 (4738, 944),
 (1344, 459),
 (3393, 529),
 (1602, 839),
 (2073, 999),
 (552, 408),
 (797, 649),
 (4518, 740),
 (2128, 148),
 (4148, 137),
 (2558, 999),
 (509, 76),
 (20, 8),
 (2351, 238),
 (1318, 542),
 (4997, 59),
 (2665, 72),
 (2540, 738),
 (4287, 967),
 (1338, 929),
 (2540, 476),
 (2540, 64),
 (599, 803),
 (2540, 527),
 (1289, 885),
 (90, 928),
 (1359, 157),
 (3715, 632),
 (3690, 240),
 (4, 338),
 (666, 208),
 (24, 152),
 (1344, 773),
 (778, 156),
 (4107, 758),
 (1241, 997),
 (1749, 403),
 (3046, 76),
 (1522, 142),
 (670, 857),
 (4107, 857),
 (282, 542),
 (3715, 526),
 (11, 59),
 (779, 187),
 (3145, 855),
 (3131, 527),
 (1681, 389),
 (3939, 974),
 (24, 16),
 (4118, 309),
 (1926, 757),
 (1543, 148),
 (582, 499),
 (3739, 334),
 (3715, 207),
 (1984, 322),
 (3021, 152),
 (1344, 884),
 (1786, 507),


In [15]:
def error_remove():
    for error in errors:
        usermovie2rating_test.pop(error)
    MSE = calc_mse(scores, usermovie2rating_test)
    print("mse:", MSE)
    pass

In [16]:
len(errors)
#great!

318

In [17]:
for error in errors:
    usermovie2rating_test.pop(error)
MSE = calc_mse(scores, usermovie2rating_test)
print("mse:", MSE)
    

mse: 0.9768210161355545
